# Reading histograms

One of the most common tasks will be translating [ROOT](https://root.cern.ch) histograms into the HEPData format. `hepdata_lib` will help you with that, and this notebook will demonstrate how to do that.

As explained in the [Getting started notebook](Getting_started.ipynb), a `Submission` needs to exist or be created. Here, we'll just create one without any additional information:

In [103]:
from hepdata_lib import Submission, Table, RootFileReader, Variable, Uncertainty
submission = Submission()
# Submission text
#submission.read_abstract("abstract.txt")
submission.add_link("Webpage with all figures and tables", "https://cms-results.web.cern.ch/cms-results/public-results/publications/SUS-21-008/")
#submission.add_link("arXiv", "Fill later")
#submission.add_record_id(1111, "inspire")

The plot will be a `Table`, in this example Figure 4a from page 12 (upper left) of the [publication](https://cms-results.web.cern.ch/cms-results/public-results/publications/B2G-17-009/index.html), which shows the distribution of the reconstructed B quark mass for the data as well as the individual simulated processes. Let's add all this, some more details as well as the actual plot (for thumbnail creation) to the `Table`:

In [104]:
import hepdata_lib
import ROOT

def extract_y_values_and_errors(file_path, is_symmetric=False):
    # Open the ROOT file
    tf = ROOT.TFile.Open(file_path)
    if not tf or tf.IsZombie():
        raise RuntimeError(f"Error: Could not open the ROOT file '{file_path}'.")

    # Create a list to hold the variables for each TGraphAsymmErrors
    variable_list = []

    # Loop over all objects in the ROOT file
    key_list = tf.GetListOfKeys()
    for key in key_list:
        obj = key.ReadObj()

        # Check if the object is a TGraphAsymmErrors
        if isinstance(obj, ROOT.TGraphAsymmErrors):
            # Extract the y-values and asymmetric errors
            y_values = [obj.GetY()[i] for i in range(obj.GetN())]
            errors_low = [obj.GetErrorYlow(i) for i in range(obj.GetN())]
            errors_high = [obj.GetErrorYhigh(i) for i in range(obj.GetN())]

            # Get the name of the TGraphAsymmErrors as the variable name
            var_name = obj.GetName()

            # Create a variable for the current TGraphAsymmErrors
            variable = hepdata_lib.Variable(var_name, values=y_values, is_independent=False, is_binned=False)

            # Create an uncertainty object to hold the asymmetric errors
            unc = hepdata_lib.Uncertainty(label=f"{var_name}_unc", is_symmetric=is_symmetric)

            # Add the asymmetric errors to the uncertainty object
            unc.values = [(error_low, error_high) for error_low, error_high in zip(errors_low, errors_high)]

            # Add the uncertainty object to the variable
            variable.add_uncertainty(unc)

            # Add the variable to the list
            variable_list.append(variable)

    # Close the ROOT file
    tf.Close()

    return variable_list


In [105]:
variable_list = extract_y_values_and_errors("hists_fig_6-8.root")
# Save the variables to a file
output_file = "variables.pkl"
with open(output_file, 'wb') as f:
    pickle.dump(variable_list, f)


loaded_variables = None
with open(output_file, 'rb') as f:
    loaded_variables = pickle.load(f)

In [135]:
def getTH1Ds(inputFile, fig, location, prefix, vars, title):
    table = Table(fig)
    table.description = title
    table.location = location
    table.keywords["cmenergies"] = ["13000"]
    table.keywords["observables"] = ["N"]
    table.keywords["phrases"] = ["Supersymmetry", "SUSY"]
    reader = RootFileReader(inputFile)
    #Data = reader.read_hist_1d(prefix+"data")
    
    for var in vars:
        readHist = reader.read_hist_1d(prefix+var)
        varHist = Variable("Number of "+var, is_independent=False, is_binned=False, units="")
        varHist.values = readHist["y"]
        table.add_variable(varHist)
    
    loaded_variables = None
    with open(output_file, 'rb') as f:
        loaded_variables = pickle.load(f) 
    print(loaded_variables)
    
    def find_variable_by_name(variable_list, name):
        for variable in variable_list:
            if variable.name == name:
                return variable
        return None
    
    v1 = find_variable_by_name(variable_list, prefix+"background_errors")
    table.add_variable(v1)
    v2 = find_variable_by_name(variable_list, prefix+"data_graph")
    table.add_variable(v2)
    #table.add_variable(prefix+"background_errors")
    #table.add_variable(prefix+"data_graph")
        
#     signalWZ1_ = reader.read_hist_1d(prefix+"signal_TChiWZ_200_180_norm")
#     prompt_vv_ = reader.read_hist_1d(prefix+"prompt_vv")
#     nonprompt_l_ = reader.read_hist_1d(prefix+"FRfakes_data")
#     prompt_dy_ = reader.read_hist_1d(prefix+"prompt_dy")
#     prompt_tt_ = reader.read_hist_1d(prefix+"prompt_tt")
#     totalBG_ = reader.read_hist_1d(prefix+"background")
    
    X = reader.read_hist_1d(prefix+vars[0])
    xBins = Variable("$p_{T}^{miss}$", is_independent=True, is_binned=True, units="GeV")
    xBins.values = X["x_edges"]
    table.add_variable(X)
    
    
    submission.add_table(table)

In [107]:
getTH1Ds(
    inputFile  = "hists_fig_6-8.root",
    fig = "Figure 6",
    location = "On Page ..",
    prefix   = "SR_2l_ewk_dM20_low_",
    vars = ["prompt_wz","prompt_vv","FRfakes_data","prompt_dy","prompt_tt","background","signal_TChiWZ_200_180_norm"],
    title    = "Postfit distribution of observed and expected yields for signal regions in the 2l soft category",
)

[<hepdata_lib.Variable object at 0x7f52664a7730>, <hepdata_lib.Variable object at 0x7f52664a7eb0>, <hepdata_lib.Variable object at 0x7f52664a78e0>, <hepdata_lib.Variable object at 0x7f52664a7f40>, <hepdata_lib.Variable object at 0x7f52664a7d90>, <hepdata_lib.Variable object at 0x7f5266864550>, <hepdata_lib.Variable object at 0x7f5266864670>, <hepdata_lib.Variable object at 0x7f5266864130>, <hepdata_lib.Variable object at 0x7f526685e250>, <hepdata_lib.Variable object at 0x7f526685e940>, <hepdata_lib.Variable object at 0x7f526685ebe0>, <hepdata_lib.Variable object at 0x7f526685e4f0>, <hepdata_lib.Variable object at 0x7f526685ed60>, <hepdata_lib.Variable object at 0x7f526685e8e0>, <hepdata_lib.Variable object at 0x7f526685e790>, <hepdata_lib.Variable object at 0x7f526685e460>, <hepdata_lib.Variable object at 0x7f526685eee0>, <hepdata_lib.Variable object at 0x7f526685e580>, <hepdata_lib.Variable object at 0x7f526615df70>, <hepdata_lib.Variable object at 0x7f526615d910>]


In [108]:
table = Table("Figure 9")
table.description = "Observed and exected yields across the search region in the 3 lepton serach in cat.A"
table.location = "Data from Figure 9, located on page 14."
table.keywords["cmenergies"] = ["13000"]
table.keywords["observables"] = ["N"]
#table.add_image("CMS-SUS-21-002_Figure_003-a.pdf")
table.keywords["phrases"] = ["Supersymmetry", "SUSY"]
#table.keywords["reactions"] = ["P P --> GRAVITON --> W+ W-", "P P --> WPRIME --> W+/W- Z0"]

The individual plot components are stored in different ROOT files, one for the individual background processes (one histogram per process plus the total), another one for the data, and the third for the signal process. All histograms here are of type [TH1](https://root.cern.ch/doc/master/classTH1.html), but you can also read in 2-dimensional [TH2](https://root.cern.ch/doc/master/classTH2.html) using `read_hist_2d(...)` instead of `read_hist_1d(...)`:

In [109]:
reader = RootFileReader("figures_9-10.root")

Data = reader.read_hist_1d("SR_3lAnew_data")
signalWZ1 = reader.read_hist_1d("SR_3lAnew_TChiWZ_200_160_norm")
signalWZ2 = reader.read_hist_1d("SR_3lAnew_TChiWZ_600_100_norm")
signalWH1 = reader.read_hist_1d("SR_3lAnew_TChiWH_150_1_norm")
signalWH2 = reader.read_hist_1d("SR_3lAnew_TChiWH_150_25_norm")
prompt_WZ_ = reader.read_hist_1d("SR_3lAnew_prompt_WZ")
prompt_VH_ = reader.read_hist_1d("SR_3lAnew_prompt_VH")
nonprompt_l_ = reader.read_hist_1d("SR_3lAnew_data_fakes")
prompt_ZZ_ = reader.read_hist_1d("SR_3lAnew_prompt_ZZ")
prompt_VVV_ = reader.read_hist_1d("SR_3lAnew_prompt_VVV")
prompt_ttX_ = reader.read_hist_1d("SR_3lAnew_prompt_TTX")
totalBG_ = reader.read_hist_1d("SR_3lAnew_total")


In [110]:
xBins = Variable("$p_{T}^{miss}$", is_independent=True, is_binned=True, units="GeV")
xBins.values = Data["x_edges"]

sigWZ1 = Variable("Number of WZ1 signal events", is_independent=False, is_binned=False, units="")
sigWZ1.values = signalWZ1["y"]
sigWZ2 = Variable("Number of WZ2 signal events", is_independent=False, is_binned=False, units="")
sigWZ2.values = signalWZ2["y"]
sigWH1 = Variable("Number of WH1 signal events", is_independent=False, is_binned=False, units="")
sigWH1.values = signalWH1["y"]
sigWH2 = Variable("Number of WH2 signal events", is_independent=False, is_binned=False, units="")
sigWH2.values = signalWH2["y"]

prompt_WZ = Variable("Number of prompt_WZ", is_independent=False, is_binned=False, units="")
prompt_VH = Variable("Number of prompt_VH", is_independent=False, is_binned=False, units="")
nonprompt_l = Variable("Number of nonprompt leptons", is_independent=False, is_binned=False, units="")
prompt_ZZ = Variable("Number of ZZ", is_independent=False, is_binned=False, units="")
prompt_VVV = Variable("Number of VVV", is_independent=False, is_binned=False, units="")
prompt_ttX = Variable("Number of ttX", is_independent=False, is_binned=False, units="")

prompt_WZ.values = prompt_WZ_["y"]
prompt_VH.values = prompt_VH_["y"]
nonprompt_l.values = nonprompt_l_["y"]
prompt_ZZ.values = prompt_ZZ_["y"]
prompt_VVV.values = prompt_VVV_["y"]
prompt_ttX.values = prompt_ttX_["y"]

totalbackground = Variable("Number of background events", is_independent=False, is_binned=False, units="")
totalbackground.values = totalBG_["y"]

data = Variable("Number of data events", is_independent=False, is_binned=False, units="")
data.values = Data["y"]

In [111]:
unc_totalbackground = Uncertainty("Total uncertainty", is_symmetric=True)
unc_totalbackground.values = totalBG_["dy"]

unc_data = Uncertainty("Data uncertainty", is_symmetric=True)
unc_data.values = Data["dy"]

totalbackground.add_uncertainty(unc_totalbackground)
data.add_uncertainty(unc_data)

In [112]:
table.add_variable(xBins)
table.add_variable(sigWZ1)
table.add_variable(sigWZ2)
table.add_variable(sigWH1)
table.add_variable(sigWH2)
table.add_variable(prompt_WZ)
table.add_variable(prompt_VH)
table.add_variable(nonprompt_l)
table.add_variable(prompt_ZZ)
table.add_variable(prompt_VVV)
table.add_variable(prompt_ttX)
table.add_variable(totalbackground)
table.add_variable(data)


submission.add_table(table)

In [113]:
def get2DScanFromText(txtfile, depUncs = [], pdf = None, qualifiers = [], obs = None, location = "", title = "", xlabel = "", ylabel = "", zlabel="", unitsx="GeV", unitsy="GeV", unitsz="", ttitle=""):
  print(txtfile)
  inPDF = pdf
  fig = Table(ttitle)
  fig.description = title
  fig.location    = location
  fig.keywords["observables"] = obs
  theDict = {"Mass1": 0, "Mass2": 1, "Expected, median":4, "Obsserved":7, "Expected, -2$\sigma$": 2, "Expected, -1$\sigma$":3, "Expected, +1$\sigma$": 5, "Expected, +2$\sigma$":6}  
  theVals = {i:[] for i in theDict.keys()}
  xvals = []
  yvals = []
  theFile = open(txtfile,"r")
  for line in theFile.readlines():
     newvals = [float(a.replace(" ", "")) for a in line.split(":")]
     if len(newvals) < 7 or any([nn > 999 for nn in newvals]): continue
     for key in theVals.keys():
        theVals[key].append(newvals[theDict[key]])
     xvals.append(newvals[theDict["Mass1"]])
     yvals.append(newvals[theDict["Mass2"]])

  ## X axis
  xvar = Variable(xlabel, is_independent=True, is_binned=False, units=unitsx)
  xvar.values = xvals
  fig.add_variable(xvar)
  yvar = Variable(ylabel, is_independent=True, is_binned=False, units=unitsy)
  yvar.values = yvals
  fig.add_variable(yvar)  
  zvars = []
  for key in theDict.keys():
       if "Mass" in key: continue 
       zvars.append(Variable(key + zlabel, is_independent=False, is_binned=False, units=unitsz))
       zvars[-1].values = theVals[key]

       for q in qualifiers:
         zvars[-1].add_qualifier(*q)
       fig.add_variable(zvars[-1])

  #fig.add_image(inPDF)
  submission.add_table(fig)


In [114]:
get2DScanFromText("summary_obs_Pure_NPFull_asymp_MLNNSOSparametriczedgeWX_WZ",
 pdf = None,
 location = "Figure 11 (top left)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "TChiWZ")

summary_obs_Pure_NPFull_asymp_MLNNSOSparametriczedgeWX_WZ


In [115]:
get2DScanFromText("summary_obs_Pure_NPFull_asymp_MLWHWX_WH",
 pdf = None,
 location = "Figure 11 (bottom left)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "TChiWH")

summary_obs_Pure_NPFull_asymp_MLWHWX_WH


In [116]:
get2DScanFromText("summary_obs_Mixed_NPFull_asymp_MLSOSparametriczedgeWXWH_WHWZ0p50",
 pdf = None,
 location = "Figure 11 (bottom right)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "TChiWZ+TChiWH")

summary_obs_Mixed_NPFull_asymp_MLSOSparametriczedgeWXWH_WHWZ0p50


In [117]:
get2DScanFromText("summary_obs_Pure_NPFull_asymp_MLNN_WZ",
 pdf = None,
 location = "Figure 12 (top)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "MLNN_WZ")

summary_obs_Pure_NPFull_asymp_MLNN_WZ


In [118]:
get2DScanFromText("summary_obs_Pure_NPFull_asymp_SOSparametric_WZ",
 pdf = None,
 location = "Figure 12 (top)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "SOSparametric_WZ")

summary_obs_Pure_NPFull_asymp_SOSparametric_WZ


In [119]:
get2DScanFromText("summary_obs_Pure_NPFull_asymp_zedge_WZ",
 pdf = None,
 location = "Figure 12 (top)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "zedge_WZ")

summary_obs_Pure_NPFull_asymp_zedge_WZ


In [120]:
get2DScanFromText("summary_obs_Pure_NPFull_asymp_WX_WZ",
 pdf = None,
 location = "Figure 12 (top)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "hadr.WX")

summary_obs_Pure_NPFull_asymp_WX_WZ


In [121]:
get2DScanFromText("summary_obs_Pure_NPFull_asymp_ML_WH",
 pdf = None,
 location = "Figure 12 (bottom)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "ML_WH")

summary_obs_Pure_NPFull_asymp_ML_WH


In [122]:
get2DScanFromText("summary_obs_Pure_NPFull_asymp_WH_WH",
 pdf = None,
 location = "Figure 12 (bottom)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "WH_WH")

summary_obs_Pure_NPFull_asymp_WH_WH


In [123]:
get2DScanFromText("summary_obs_Pure_NPFull_asymp_WX_WH",
 pdf = None,
 location = "Figure 12 (bottom)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "Hadr_WH")

summary_obs_Pure_NPFull_asymp_WX_WH


In [124]:
get2DScanFromText("summary_obs_Mixed_NPFull_asymp_MLzedge4b_HHHZZZ1p00",
 pdf = None,
 location = "Figure 13 (top left)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "ZZ")

summary_obs_Mixed_NPFull_asymp_MLzedge4b_HHHZZZ1p00


In [125]:
get2DScanFromText("summary_obs_Mixed_NPFull_asymp_MLzedge4b_HHHZZZ0p00",
 pdf = None,
 location = "Figure 13 (top right)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "HH")

summary_obs_Mixed_NPFull_asymp_MLzedge4b_HHHZZZ0p00


In [126]:
get2DScanFromText("summary_obs_Mixed_NPFull_asymp_MLzedge4b_HHHZZZ0p50",
 pdf = None,
 location = "Figure 13 (bottom)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "ZH")

summary_obs_Mixed_NPFull_asymp_MLzedge4b_HHHZZZ0p50


In [127]:
get2DScanFromText("summary_byBR",
 pdf = None,
 location = "Figure 14",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "BR Combined limit")

summary_byBR


In [128]:
get2DScanFromText("summary_byBR_ML",
 pdf = None,
 location = "Figure 14",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "ML Limit")

summary_byBR_ML


In [129]:
get2DScanFromText("summary_byBR_zedge",
 pdf = None,
 location = "Figure 14",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "Zedge Limit")

summary_byBR_zedge


In [130]:
get2DScanFromText("summary_byBR_4b",
 pdf = None,
 location = "Figure 14",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "4b Limit")

summary_byBR_4b


In [131]:
get2DScanFromText("summary_obs_Pure_NPFull_asymp_WX4bMLWH_WWWHHHhino",
 pdf = None,
 location = "Figure 15",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "Higgsino combined limit")

summary_obs_Pure_NPFull_asymp_WX4bMLWH_WWWHHHhino


In [132]:
get2DScanFromText("summary_obs_Pure_NPFull_asymp_SOSparametric_slep",
 pdf = None,
 location = "Figure 16 (left)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "SOS Combined limit")

summary_obs_Pure_NPFull_asymp_SOSparametric_slep


In [133]:
get2DScanFromText("summary_obs_Pure_NPFull_asymp_SOSparametriczedge_slep",
 pdf = None,
 location = "Figure 16 (right)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "Compressed SOS limit")

summary_obs_Pure_NPFull_asymp_SOSparametriczedge_slep


In [134]:
submission.create_files("Test",remove_old=True)